In [37]:
import pandas as pd
from datetime import datetime
import numpy as np


In [38]:
# colunas que vao para o arquivo FULL final
columns = [
    "timestamp",
    "THROTTLE",
    "BRAKE",
    "ECU_MODE_ID",
    "TORQUE_GAIN",
    "TORQUE_REF_LEFT_MOTOR",
    "TORQUE_REF_RIGHT_MOTOR",
    "LEFT_MOTOR_SPEED",
    "RIGHT_MOTOR_SPEED",
    "ACCEL_LONGITUDINAL",
    "STACK_1_CELL_1",
    "STACK_1_CELL_2",
    "STACK_1_CELL_3",
    "STACK_1_CELL_4",
    "STACK_2_CELL_1",
    "STACK_2_CELL_2",
    "STACK_2_CELL_3",
    "STACK_2_CELL_4",
    "STACK_3_CELL_1",
    "STACK_3_CELL_2",
    "STACK_3_CELL_3",
    "STACK_3_CELL_4",
    "STACK_4_CELL_1",
    "STACK_4_CELL_2",
    "STACK_4_CELL_3",
    "STACK_4_CELL_4",
    "STACK_5_CELL_1",
    "STACK_5_CELL_2",
    "STACK_5_CELL_3",
    "STACK_5_CELL_4",
    "STACK_6_CELL_1",
    "STACK_6_CELL_2",
    "STACK_6_CELL_3",
    "STACK_6_CELL_4",
    "MAX_VOLTAGE",
    "MIN_VOLTAGE",
    "TOTAL_VOLTAGE",
    "SHUNT_CURRENT",
    "BMS_MODE_ID",
    "BMS_ERROR_ID",
    "AIR_P",
    "AIR_N"
]

In [39]:
# timestamps de interesse
# nome: (inicio, fim)
interest_timestamps = {
    # miguelito
    "aceleracao1_miguelito": ("2025-12-08 15:38:00", "2025-12-08 15:38:30"), # aceleração bem leve, estranho
    "autocross1_miguelito":  ("2025-12-08 15:28:25", "2025-12-08 15:29:00"), # incompleto, shunt ruim
    "autocross2_miguelito":  ("2025-12-08 15:39:20", "2025-12-08 15:40:00"), # OK
    "autocross3_miguelito":  ("2025-12-08 15:43:30", "2025-12-08 15:45:30"), # OK
    "aceleracao2_miguelito": ("2025-12-08 15:47:40", "2025-12-08 15:48:00"), # com falha de subtensão
    "autocross4_miguelito": ("2025-12-08 15:49:00", "2025-12-08 15:50:00"), # OK
    "aceleracao3_miguelito": ("2025-12-08 16:13:15", "2025-12-08 16:13:35"), # bom
    "autocross5_miguelito": ("2025-12-08 16:16:00", "2025-12-08 16:17:30"), # OK
    "aceleracao4_miguelito": ("2025-12-08 16:21:40", "2025-12-08 16:22:15"), # com falha de subtensão
    "aceleracao5_miguelito": ("2025-12-08 16:39:00", "2025-12-08 16:39:30"), # com falha de subtensão

    # mike
    "aceleracao1_mike": ("2025-12-08 17:01:00", "2025-12-08 17:01:20"), # com falha de subtensão
    "autocross1_mike": ("2025-12-08 17:03:00", "2025-12-08 17:04:15"), 
    "aceleracao2_mike": ("2025-12-08 17:07:20", "2025-12-08 17:07:45"), # com falha de subtensão
    "aceleracao3_mike": ("2025-12-08 17:10:40", "2025-12-08 17:11:05"), # 50 km/h, publico
    "autocross2_mike": ("2025-12-08 17:21:00", "2025-12-08 17:24:30"), 
    "aceleracao4_mike": ("2025-12-08 17:26:45", "2025-12-08 17:27:00"), # com falha de subtensão
    "autocross3_mike": ("2025-12-08 17:28:15", "2025-12-08 17:31:15"), 
    "aceleracao5_mike": ("2025-12-08 17:33:45", "2025-12-08 17:34:00"), # com falha de subtensão
    "aceleracao6_mike": ("2025-12-08 17:40:50", "2025-12-08 17:41:10"), # com falha de subtensão

    # pedro
    "autocross1_pedro": ("2025-12-08 18:01:45", "2025-12-08 18:04:30"), # tensoes estranhas nas stacks
    "aceleracao1_pedro": ("2025-12-08 18:16:25", "2025-12-08 18:16:45"), # com falha de subtensão
}

In [40]:
full_df = pd.DataFrame([], columns=columns)
full_df

,timestamp,THROTTLE,BRAKE,ECU_MODE_ID,TORQUE_GAIN,TORQUE_REF_LEFT_MOTOR,TORQUE_REF_RIGHT_MOTOR,LEFT_MOTOR_SPEED,RIGHT_MOTOR_SPEED,ACCEL_LONGITUDINAL,...,STACK_6_CELL_3,STACK_6_CELL_4,MAX_VOLTAGE,MIN_VOLTAGE,TOTAL_VOLTAGE,SHUNT_CURRENT,BMS_MODE_ID,BMS_ERROR_ID,AIR_P,AIR_N


In [41]:
file_names = [
    "76_DRIVER_INPUTS.csv",
    "77_ECU_MODE.csv",
    "78_TORQUE_REFERENCE.csv",
    "79_WHEEL_SPEED.csv",
    "80_MOTOR_SPEED.csv",
    "85_LEFT_MOTOR_TRACTIVE.csv",
    "95_RIGHT_MOTOR_TRACTIVE.csv",
    "259_ACCELEROMETER.csv",
    "260_GYROSCOPE.csv",
    "261_ELETROBUILD_TEMPERATURE.csv",
    "300_STACK_1_VOLTAGE.csv",
    "301_STACK_2_VOLTAGE.csv",
    "302_STACK_3_VOLTAGE.csv",
    "303_STACK_4_VOLTAGE.csv",
    "304_STACK_5_VOLTAGE.csv",
    "305_STACK_6_VOLTAGE.csv",
    "306_ACCUMULADOR_PARAMS.csv",
    "307_BMS_PARAMS.csv",
]

folder_names = [
    "./data/dados_telemetria/teste_odonto_08122025/teste1_0812",
    "./data/dados_telemetria/teste_odonto_08122025/teste2_0812",
    "./data/dados_telemetria/teste_odonto_08122025/teste4_0812",
    "./data/dados_telemetria/teste_odonto_08122025/teste5_0812",
    "./data/dados_telemetria/teste_odonto_08122025/teste7_longer",
    "./data/dados_telemetria/teste_odonto_08122025/teste8",
]

stacks_list = [
    'STACK_1_CELL_1', 'STACK_1_CELL_2', 'STACK_1_CELL_3', 'STACK_1_CELL_4',
    'STACK_2_CELL_1', 'STACK_2_CELL_2', 'STACK_2_CELL_3', 'STACK_2_CELL_4',
    'STACK_3_CELL_1', 'STACK_3_CELL_2', 'STACK_3_CELL_3', 'STACK_3_CELL_4',
    'STACK_4_CELL_1', 'STACK_4_CELL_2', 'STACK_4_CELL_3', 'STACK_4_CELL_4',
    'STACK_5_CELL_1', 'STACK_5_CELL_2', 'STACK_5_CELL_3', 'STACK_5_CELL_4',
    'STACK_6_CELL_1', 'STACK_6_CELL_2', 'STACK_6_CELL_3', 'STACK_6_CELL_4',
]


In [42]:
full_folder = "./data/dados_telemetria/teste_odonto_08122025/full"

for file_name in file_names:
    print(file_name + "...")
    dfs_list = []
    for folder in folder_names:
        dfs_list.append(pd.read_csv(f"{folder}/{file_name}"))

    combined_df = pd.concat(dfs_list, ignore_index=True)
    combined_df.to_csv(f"{full_folder}/{"FULL_" + file_name}", index=False)

76_DRIVER_INPUTS.csv...
77_ECU_MODE.csv...
78_TORQUE_REFERENCE.csv...
79_WHEEL_SPEED.csv...
80_MOTOR_SPEED.csv...
85_LEFT_MOTOR_TRACTIVE.csv...
95_RIGHT_MOTOR_TRACTIVE.csv...
259_ACCELEROMETER.csv...
260_GYROSCOPE.csv...
261_ELETROBUILD_TEMPERATURE.csv...
300_STACK_1_VOLTAGE.csv...
301_STACK_2_VOLTAGE.csv...
302_STACK_3_VOLTAGE.csv...
303_STACK_4_VOLTAGE.csv...
304_STACK_5_VOLTAGE.csv...
305_STACK_6_VOLTAGE.csv...
306_ACCUMULADOR_PARAMS.csv...
307_BMS_PARAMS.csv...


In [43]:
# agora junta as que nao estao no inicial
extra_file_names = [
    "311_STACK_OVER_ERROR.csv",
    "312_STACK_ERROR_UNDER.csv",
]

extra_folder_names = [
    "./data/dados_telemetria/teste_odonto_08122025/teste5_0812",
    "./data/dados_telemetria/teste_odonto_08122025/teste7_longer",
]

full_folder = "./data/dados_telemetria/teste_odonto_08122025/full"

for file_name in extra_file_names:
    print(file_name + "...")
    dfs_list = []
    for folder in extra_folder_names:
        dfs_list.append(pd.read_csv(f"{folder}/{file_name}"))

    combined_df = pd.concat(dfs_list, ignore_index=True)
    combined_df.to_csv(f"{full_folder}/{"FULL_" + file_name}", index=False)

311_STACK_OVER_ERROR.csv...
312_STACK_ERROR_UNDER.csv...


In [44]:
# quais colunas precisa refatorar para colocar no arquivo final FULL.csv
# (old_name, new_name)
columns_rename_map = {
    "FULL_77_ECU_MODE.csv": [("MODE_ID", "ECU_MODE_ID"), ("TORQUE_GAIN_ID", "TORQUE_GAIN")],
    "FULL_85_LEFT_MOTOR_TRACTIVE.csv": [("SPEED", "LEFT_MOTOR_SPEED")],
    "FULL_95_RIGHT_MOTOR_TRACTIVE.csv": [("SPEED", "RIGHT_MOTOR_SPEED")],
    "FULL_259_ACCELEROMETER.csv": [("ACCEL_Z", "ACCEL_LONGITUDINAL")],
    "FULL_300_STACK_1_VOLTAGE.csv": [
        ("CEL_1", "STACK_1_CELL_1"), ("CEL_2", "STACK_1_CELL_2"), ("CEL_3", "STACK_1_CELL_3"), ("CEL_4", "STACK_1_CELL_4")
    ],
    "FULL_301_STACK_2_VOLTAGE.csv": [
        ("CEL_1", "STACK_2_CELL_1"), ("CEL_2", "STACK_2_CELL_2"), ("CEL_3", "STACK_2_CELL_3"), ("CEL_4", "STACK_2_CELL_4")
    ],
    "FULL_302_STACK_3_VOLTAGE.csv": [
        ("CEL_1", "STACK_3_CELL_1"), ("CEL_2", "STACK_3_CELL_2"), ("CEL_3", "STACK_3_CELL_3"), ("CEL_4", "STACK_3_CELL_4")
    ],
    "FULL_303_STACK_4_VOLTAGE.csv": [
        ("CEL_1", "STACK_4_CELL_1"), ("CEL_2", "STACK_4_CELL_2"), ("CEL_3", "STACK_4_CELL_3"), ("CEL_4", "STACK_4_CELL_4")
    ],
    "FULL_304_STACK_5_VOLTAGE.csv": [
        ("CEL_1", "STACK_5_CELL_1"), ("CEL_2", "STACK_5_CELL_2"), ("CEL_3", "STACK_5_CELL_3"), ("CEL_4", "STACK_5_CELL_4")
    ],
    "FULL_305_STACK_6_VOLTAGE.csv": [
        ("CEL_1", "STACK_6_CELL_1"), ("CEL_2", "STACK_6_CELL_2"), ("CEL_3", "STACK_6_CELL_3"), ("CEL_4", "STACK_6_CELL_4")
    ],
    "FULL_306_ACCUMULADOR_PARAMS.csv": [("SHUNT_VOLTAGE", "SHUNT_CURRENT")],
    "FULL_307_BMS_PARAMS.csv": [("MODE_ID", "BMS_MODE_ID"), ("ERROR_ID", "BMS_ERROR_ID")],
}

# quais colunas vou efetivamente usar dos full individuais - nomes refatorados ja
file_columns_map = {
    'FULL_76_DRIVER_INPUTS.csv': ['THROTTLE', 'BRAKE'],
    'FULL_77_ECU_MODE.csv': ['ECU_MODE_ID', 'TORQUE_GAIN'],
    'FULL_78_TORQUE_REFERENCE.csv': ['TORQUE_REF_LEFT_MOTOR', 'TORQUE_REF_RIGHT_MOTOR'],
    'FULL_85_LEFT_MOTOR_TRACTIVE.csv': ['LEFT_MOTOR_SPEED'],
    'FULL_95_RIGHT_MOTOR_TRACTIVE.csv': ['RIGHT_MOTOR_SPEED'],
    'FULL_259_ACCELEROMETER.csv': ['ACCEL_LONGITUDINAL'],
    'FULL_300_STACK_1_VOLTAGE.csv': ['STACK_1_CELL_1', 'STACK_1_CELL_2', 'STACK_1_CELL_3', 'STACK_1_CELL_4'],
    'FULL_301_STACK_2_VOLTAGE.csv': ['STACK_2_CELL_1', 'STACK_2_CELL_2', 'STACK_2_CELL_3', 'STACK_2_CELL_4'],
    'FULL_302_STACK_3_VOLTAGE.csv': ['STACK_3_CELL_1', 'STACK_3_CELL_2', 'STACK_3_CELL_3', 'STACK_3_CELL_4'],
    'FULL_303_STACK_4_VOLTAGE.csv': ['STACK_4_CELL_1', 'STACK_4_CELL_2', 'STACK_4_CELL_3', 'STACK_4_CELL_4'],
    'FULL_304_STACK_5_VOLTAGE.csv': ['STACK_5_CELL_1', 'STACK_5_CELL_2', 'STACK_5_CELL_3', 'STACK_5_CELL_4'],
    'FULL_305_STACK_6_VOLTAGE.csv': ['STACK_6_CELL_1', 'STACK_6_CELL_2', 'STACK_6_CELL_3', 'STACK_6_CELL_4'],
    'FULL_306_ACCUMULADOR_PARAMS.csv': ['MAX_VOLTAGE', 'MIN_VOLTAGE', 'TOTAL_VOLTAGE', 'SHUNT_CURRENT'],
    'FULL_307_BMS_PARAMS.csv': ['BMS_MODE_ID', 'BMS_ERROR_ID', 'AIR_P', 'AIR_N'],
}

In [45]:
# passa pelos fulls: remove timestamps que estão fora dos ranges
def is_in_range(timestamp):
    for key in interest_timestamps:
        start_date, end_date = interest_timestamps[key]

        start_timestamp = datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000
        end_timestamp = datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000

        if timestamp >= start_timestamp and timestamp <= end_timestamp:
            return True, key
        
    return False, None

full_folder = "./data/dados_telemetria/teste_odonto_08122025/full"
# full_file_names = file_names + extra_file_names
full_file_names = file_names

for i in range(len(full_file_names)):
    full_file_names[i] = "FULL_" + full_file_names[i]

for file_name in full_file_names:
    print(file_name + "...")

    if file_name == 'FULL_300_STACK_1_VOLTAGE.csv':
        print("here")

    # verifica se existe na lista do file_columns_map
    if file_name not in file_columns_map:
        continue

    # le o arquivo atual
    df = pd.read_csv(f'{full_folder}/{file_name}')

    # monta o dicionario de dados desse arquivo
    data_row = {}

    for column in columns:
        data_row[column] = []

    # varre os timestamps
    for i in range (df.shape[0]):
        curr_timestamp = df.iloc[i, 1] 
        is_range, key = is_in_range(timestamp=curr_timestamp)
        
        if is_range:
            # pega as colunas desse arquivo
            columns_to_add = file_columns_map[file_name]

            # para cada coluna do full_df
            for col in full_df.columns:
                if col == 'timestamp': 
                    data_row['timestamp'].append(curr_timestamp)
                    continue 

                # verifica se essa coluna tem dados para adicionar
                if col in columns_to_add:
                    # se o arquivo atual esta na lista de refatoração
                    if file_name in columns_rename_map:
                        refactor_list = columns_rename_map[file_name]
                        result = next((item for item in refactor_list if item[1] == col), None)

                        if result == None:
                            # a coluna atual nao precisa de refatoração
                            data_row[col].append(df.iloc[i][col])
                            continue
                        
                        old_name, new_name = result
                        data_row[new_name].append(df.iloc[i][old_name])
                    else:
                        data_row[col].append(df.iloc[i][col])
                else:
                    # sem dados para adicionar
                    data_row[col].append(np.nan)

    # salva o dicionario na full_df
    new_rows = pd.DataFrame(data_row)
    full_df = pd.concat([full_df, new_rows], ignore_index=True)

# ordena o df por timestamp
sorted_df = full_df.sort_values(by='timestamp')

# agrupa e toma a media de timestamps duplicados
grouped_df = sorted_df.groupby('timestamp', as_index=False).mean()

# zera as medições NA da primeira linha
for j in range(grouped_df.shape[1]):
    if pd.isna(grouped_df.iloc[0, j]):
        grouped_df.iloc[0, j] = 0

# remover os NA puxando o valor do timestamp anterior
filled_df = grouped_df.fillna(method='ffill')

TRANSMISSION_RATIO = 8.89
WHEEL_RADIUS = 20.5 * 2.54 / 2 / 100
PI = 3.1415
MS_TO_KPH = 3.6
SHUNT_OFFSET = 45
ACCEL_SENSITIVITY_FACTOR = 0.122 / 1000

# formata grandezas fisica
filled_df['t'] = filled_df['timestamp'] - filled_df['timestamp'][0]
filled_df['LEFT_MOTOR_SPEED'] = filled_df['LEFT_MOTOR_SPEED'] / TRANSMISSION_RATIO * WHEEL_RADIUS * 2 * PI / 60 * MS_TO_KPH
filled_df['RIGHT_MOTOR_SPEED'] = filled_df['RIGHT_MOTOR_SPEED'] / TRANSMISSION_RATIO * WHEEL_RADIUS * 2 * PI / 60 * MS_TO_KPH
filled_df['ACCEL_LONGITUDINAL'] = filled_df['ACCEL_LONGITUDINAL'] * ACCEL_SENSITIVITY_FACTOR
filled_df['SHUNT_CURRENT'] = filled_df['SHUNT_CURRENT'] - SHUNT_OFFSET

# remove dados de ruidos / absurdos
for i in range(1, filled_df.shape[0]):
    if filled_df.loc[i, 'THROTTLE'] > 1500 or filled_df.loc[i, 'THROTTLE'] < 0:
        filled_df.loc[i, 'THROTTLE'] = filled_df.loc[i-1, 'THROTTLE']

    if filled_df.loc[i, 'BRAKE'] > 100 or filled_df.loc[i, 'BRAKE'] < 0:
        filled_df.loc[i, 'BRAKE'] = filled_df.loc[i-1, 'BRAKE']

    if filled_df.loc[i, 'ECU_MODE_ID'] > 10 or filled_df.loc[i, 'ECU_MODE_ID'] < 0:
        filled_df.loc[i, 'ECU_MODE_ID'] = filled_df.loc[i-1, 'ECU_MODE_ID']

    if filled_df.loc[i, 'TORQUE_GAIN'] > 1000 or filled_df.loc[i, 'TORQUE_GAIN'] < 0:
        filled_df.loc[i, 'TORQUE_GAIN'] = filled_df.loc[i-1, 'TORQUE_GAIN']

    if filled_df.loc[i, 'TORQUE_REF_LEFT_MOTOR'] > 5000 or filled_df.loc[i, 'TORQUE_REF_LEFT_MOTOR'] < 0:
        filled_df.loc[i, 'TORQUE_REF_LEFT_MOTOR'] = filled_df.loc[i-1, 'TORQUE_REF_LEFT_MOTOR']

    if filled_df.loc[i, 'TORQUE_REF_RIGHT_MOTOR'] > 5000 or filled_df.loc[i, 'TORQUE_REF_RIGHT_MOTOR'] < 0:
        filled_df.loc[i, 'TORQUE_REF_RIGHT_MOTOR'] = filled_df.loc[i-1, 'TORQUE_REF_RIGHT_MOTOR']

    if filled_df.loc[i, 'LEFT_MOTOR_SPEED'] > 100 or filled_df.loc[i, 'LEFT_MOTOR_SPEED'] < 0:
        filled_df.loc[i, 'LEFT_MOTOR_SPEED'] = filled_df.loc[i-1, 'LEFT_MOTOR_SPEED']

    if filled_df.loc[i, 'RIGHT_MOTOR_SPEED'] > 100 or filled_df.loc[i, 'RIGHT_MOTOR_SPEED'] < 0:
        filled_df.loc[i, 'RIGHT_MOTOR_SPEED'] = filled_df.loc[i-1, 'RIGHT_MOTOR_SPEED']

    if filled_df.loc[i, 'ACCEL_LONGITUDINAL'] > 10 or filled_df.loc[i, 'ACCEL_LONGITUDINAL'] < -10:
        filled_df.loc[i, 'ACCEL_LONGITUDINAL'] = filled_df.loc[i-1, 'ACCEL_LONGITUDINAL']

    for stack in stacks_list:
        if filled_df.loc[i, stack] > 10 or filled_df.loc[i, stack] < -10:
            filled_df.loc[i, stack] = filled_df.loc[i-1, stack]

    if filled_df.loc[i, 'MAX_VOLTAGE'] > 10 or filled_df.loc[i, 'MAX_VOLTAGE'] < -10:
        filled_df.loc[i, 'MAX_VOLTAGE'] = filled_df.loc[i-1, 'MAX_VOLTAGE']

    if filled_df.loc[i, 'MIN_VOLTAGE'] > 10 or filled_df.loc[i, 'MIN_VOLTAGE'] < -10:
        filled_df.loc[i, 'MIN_VOLTAGE'] = filled_df.loc[i-1, 'MIN_VOLTAGE']

    if filled_df.loc[i, 'TOTAL_VOLTAGE'] > 200 or filled_df.loc[i, 'TOTAL_VOLTAGE'] < -200:
        filled_df.loc[i, 'TOTAL_VOLTAGE'] = filled_df.loc[i-1, 'TOTAL_VOLTAGE']

    if filled_df.loc[i, 'SHUNT_CURRENT'] > 800 or filled_df.loc[i, 'SHUNT_CURRENT'] < -800:
        filled_df.loc[i, 'SHUNT_CURRENT'] = filled_df.loc[i-1, 'SHUNT_CURRENT']

    if filled_df.loc[i, 'BMS_MODE_ID'] > 50 or filled_df.loc[i, 'BMS_MODE_ID'] < 0:
        filled_df.loc[i, 'BMS_MODE_ID'] = filled_df.loc[i-1, 'BMS_MODE_ID']

    if filled_df.loc[i, 'BMS_ERROR_ID'] > 50 or filled_df.loc[i, 'BMS_ERROR_ID'] < 0:
        filled_df.loc[i, 'BMS_ERROR_ID'] = filled_df.loc[i-1, 'BMS_ERROR_ID']

    if filled_df.loc[i, 'AIR_P'] > 10 or filled_df.loc[i, 'AIR_P'] < 0:
        filled_df.loc[i, 'AIR_P'] = filled_df.loc[i-1, 'AIR_P']

    if filled_df.loc[i, 'AIR_N'] > 10 or filled_df.loc[i, 'AIR_N'] < 0:
        filled_df.loc[i, 'AIR_N'] = filled_df.loc[i-1, 'AIR_N']

# faz o split e salva em cada CSV
for key in interest_timestamps:
    folder = ""

    if "miguelito" in key: folder = "miguelito"
    if "mike" in key: folder = "mike"
    if "pedro" in key: folder = "pedro"

    start_date, end_date = interest_timestamps[key]

    start_timestamp = datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000
    end_timestamp = datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000

    splitted_df = filled_df[(filled_df["timestamp"] >= start_timestamp) & (filled_df["timestamp"] < end_timestamp)]

    splitted_df.to_csv(f'{full_folder}/odonto_08122025/{folder}/{key}.csv', index=False)


# salva o df final para um CSV novo
filled_df.to_csv(f'{full_folder}/full_df.csv', index=False)

FULL_76_DRIVER_INPUTS.csv...
FULL_77_ECU_MODE.csv...
FULL_78_TORQUE_REFERENCE.csv...
FULL_79_WHEEL_SPEED.csv...
FULL_80_MOTOR_SPEED.csv...
FULL_85_LEFT_MOTOR_TRACTIVE.csv...
FULL_95_RIGHT_MOTOR_TRACTIVE.csv...
FULL_259_ACCELEROMETER.csv...
FULL_260_GYROSCOPE.csv...
FULL_261_ELETROBUILD_TEMPERATURE.csv...
FULL_300_STACK_1_VOLTAGE.csv...
here


C:\Users\rapha\AppData\Local\Temp\ipykernel_8212\3975095963.py:77: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_df = pd.concat([full_df, new_rows], ignore_index=True)


FULL_301_STACK_2_VOLTAGE.csv...
FULL_302_STACK_3_VOLTAGE.csv...
FULL_303_STACK_4_VOLTAGE.csv...
FULL_304_STACK_5_VOLTAGE.csv...
FULL_305_STACK_6_VOLTAGE.csv...
FULL_306_ACCUMULADOR_PARAMS.csv...
FULL_307_BMS_PARAMS.csv...


C:\Users\rapha\AppData\Local\Temp\ipykernel_8212\3975095963.py:91: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  filled_df = grouped_df.fillna(method='ffill')
C:\Users\rapha\AppData\Local\Temp\ipykernel_8212\3975095963.py:91: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  filled_df = grouped_df.fillna(method='ffill')
